```
This part takes computing power, so we run it in google's Colab using its free GPU.
```

### Mount G-drive

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
cd /

In [ ]:
!mkdir -p /drive
!google-drive-ocamlfuse drive

In [ ]:
ls

In [ ]:
cd /content

In [ ]:
# Move the zipped file folder to /content/
!cp drive/xxxxxx.zip /content/
!unzip xxxxxx.zip

---

In [ ]:
# download model
!wget https://www.dropbox.com/s/llpxd14is7gyj0z/model.h5
# download script modules  
!wget https://raw.githubusercontent.com/michalfaber/keras_Realtime_Multi-Person_Pose_Estimation/master/util.py
!wget https://raw.githubusercontent.com/michalfaber/keras_Realtime_Multi-Person_Pose_Estimation/master/config_reader.py
!wget https://raw.githubusercontent.com/michalfaber/keras_Realtime_Multi-Person_Pose_Estimation/master/config
!wget https://raw.githubusercontent.com/michalfaber/keras_Realtime_Multi-Person_Pose_Estimation/master/model.py

!pip install configobj

In [0]:
import cv2
import math
import time
import numpy as np
import util
from config_reader import config_reader
from scipy.ndimage.filters import gaussian_filter
from model import get_testing_model
import matplotlib.pyplot as plt
import os
import pandas as pd
import scipy.io as sio

# find connection in the specified sequence, center 29 is in the position 15
limbSeq = [[2, 3], [2, 6], [3, 4], [4, 5], [6, 7], [7, 8], [2, 9], [9, 10],
           [10, 11], [2, 12], [12, 13], [13, 14], [2, 1], [1, 15], [15, 17],
           [1, 16], [16, 18], [3, 17], [6, 18]]

# the middle joints heatmap correpondence
mapIdx = [[31, 32], [39, 40], [33, 34], [35, 36], [41, 42], [43, 44], [19, 20], [21, 22],
          [23, 24], [25, 26], [27, 28], [29, 30], [
              47, 48], [49, 50], [53, 54], [51, 52],
          [55, 56], [37, 38], [45, 46]]

# visualize
colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0],
          [0, 255, 0],
          [0, 255, 85], [0, 255, 170], [0, 255, 255], [
              0, 170, 255], [0, 85, 255], [0, 0, 255],
          [85, 0, 255],
          [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85]]

In [0]:
def process(input_image, params, model_params):

    oriImg = input_image  # B,G,R order
    multiplier = [x * model_params['boxsize'] / oriImg.shape[0]
                  for x in params['scale_search']]

    heatmap_avg = np.zeros((oriImg.shape[0], oriImg.shape[1], 19))
    paf_avg = np.zeros((oriImg.shape[0], oriImg.shape[1], 38))

    for m in range(len(multiplier)):
        scale = multiplier[m]

        imageToTest = cv2.resize(
            oriImg, (0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
        imageToTest_padded, pad = util.padRightDownCorner(imageToTest, model_params['stride'],
                                                          model_params['padValue'])

        # required shape (1, width, height, channels)
        input_img = np.transpose(np.float32(
            imageToTest_padded[:, :, :, np.newaxis]), (3, 0, 1, 2))

        output_blobs = model.predict(input_img)

        # extract outputs, resize, and remove padding
        heatmap = np.squeeze(output_blobs[1])  # output 1 is heatmaps
        heatmap = cv2.resize(heatmap, (0, 0), fx=model_params['stride'], fy=model_params['stride'],
                             interpolation=cv2.INTER_CUBIC)
        heatmap = heatmap[:imageToTest_padded.shape[0] - pad[2], :imageToTest_padded.shape[1] - pad[3],
                          :]
        heatmap = cv2.resize(heatmap, (oriImg.shape[1], oriImg.shape[
                             0]), interpolation=cv2.INTER_CUBIC)

        paf = np.squeeze(output_blobs[0])  # output 0 is PAFs
        paf = cv2.resize(paf, (0, 0), fx=model_params['stride'], fy=model_params['stride'],
                         interpolation=cv2.INTER_CUBIC)
        paf = paf[:imageToTest_padded.shape[0] - pad[2],
                  :imageToTest_padded.shape[1] - pad[3], :]
        paf = cv2.resize(paf, (oriImg.shape[1], oriImg.shape[
                         0]), interpolation=cv2.INTER_CUBIC)

        heatmap_avg = heatmap_avg + heatmap / len(multiplier)
        paf_avg = paf_avg + paf / len(multiplier)

    all_peaks = []
    peak_counter = 0

    for part in range(18):
        map_ori = heatmap_avg[:, :, part]
        map = gaussian_filter(map_ori, sigma=3)

        map_left = np.zeros(map.shape)
        map_left[1:, :] = map[:-1, :]
        map_right = np.zeros(map.shape)
        map_right[:-1, :] = map[1:, :]
        map_up = np.zeros(map.shape)
        map_up[:, 1:] = map[:, :-1]
        map_down = np.zeros(map.shape)
        map_down[:, :-1] = map[:, 1:]

        peaks_binary = np.logical_and.reduce(
            (map >= map_left, map >= map_right, map >= map_up, map >= map_down, map > params['thre1']))
        peaks = list(zip(np.nonzero(peaks_binary)[1], np.nonzero(
            peaks_binary)[0]))  # note reverse
        peaks_with_score = [x + (map_ori[x[1], x[0]],) for x in peaks]
        id = range(peak_counter, peak_counter + len(peaks))
        peaks_with_score_and_id = [peaks_with_score[
            i] + (id[i],) for i in range(len(id))]

        all_peaks.append(peaks_with_score_and_id)
        peak_counter += len(peaks)

    coco2lsp = [10, 9, 8, 11, 12, 13, 4, 3, 2, 5, 6, 7, 1]
    pred_lsp = np.zeros((13, 2))
    for j in range(len(coco2lsp)):
        id = coco2lsp[j]
        if all_peaks[id]:
            if len(all_peaks[id]) > 1:
                multi_peaks = []
                for m in range(0, len(all_peaks[id])):
                    multi_peaks.append(list(all_peaks[id][m]))
                prob_multi_peaks = [x[2] for x in multi_peaks]
                peak_prob_index = prob_multi_peaks.index(
                    max(prob_multi_peaks))
                pred_lsp[j, 0] = all_peaks[id][peak_prob_index][0]
                pred_lsp[j, 1] = all_peaks[id][peak_prob_index][1]
            else:
                pred_lsp[j, 0] = all_peaks[id][0][0]  # x
                pred_lsp[j, 1] = all_peaks[id][0][1]  # y

    return pred_lsp

In [35]:
keras_weights_file = 'model.h5'
model = get_testing_model()
model.load_weights(keras_weights_file)

path = 'popping99/'
csv_path = 'popping99_csv/'
if not os.path.exists(csv_path):
    os.makedirs(csv_path)

for dance in range(20, 107):
  video = str(dance) + '.mp4'
  print('current video: ' + video)
  
  videoCapture = cv2.VideoCapture()
  videoCapture.open(path + video)
  fps = videoCapture.get(cv2.CAP_PROP_FPS)
  frames = videoCapture.get(cv2.CAP_PROP_FRAME_COUNT)
  print(frames)

  pred = np.zeros((int(frames), 13, 2))
  for i in range(int(frames)):
    ret, frame = videoCapture.read()
    input_image = frame
    tic = time.time()
    print('frame: ' + str(i))

    # load config
    params, model_params = config_reader()

    # generate image with body parts
    pred_lsp = process(input_image, params, model_params)
    pred[i] = pred_lsp
    toc = time.time()
    print('processing time is %.5f' % (toc - tic))
  print(pred)
  index = pd.MultiIndex.from_product([range(pred.shape[0]),range(pred.shape[1])],names=['frame', 'keypoint'])
  df = pd.DataFrame(dict(x=np.reshape(pred[:, :, 0],-1), y=np.reshape(pred[:, :, 1],-1)), index=index)
  df.to_csv(csv_path + video.split('.')[0] + '.csv')
  pred_dict = {'pred': pred}
  sio.savemat(csv_path + video.split('.')[0] + '.mat', pred_dict)

current video: 20.mp4
50.0
frame: 0
processing time is 6.43480
frame: 1
processing time is 5.14794
frame: 2
processing time is 5.15348
frame: 3
processing time is 5.15814
frame: 4
processing time is 5.18639
frame: 5
processing time is 5.18556
frame: 6
processing time is 5.20224
frame: 7
processing time is 5.16947
frame: 8
processing time is 5.14733
frame: 9
processing time is 5.17343
frame: 10
processing time is 5.15148
frame: 11
processing time is 5.17435
frame: 12
processing time is 5.13349
frame: 13
processing time is 5.16369
frame: 14
processing time is 5.17840
frame: 15
processing time is 5.15795
frame: 16
processing time is 5.16157
frame: 17
processing time is 5.17767
frame: 18
processing time is 5.17857
frame: 19
processing time is 5.18624
frame: 20
processing time is 5.17149
frame: 21
processing time is 5.16230
frame: 22
processing time is 5.16262
frame: 23
processing time is 5.17768
frame: 24
processing time is 5.16525
frame: 25
processing time is 5.15296
frame: 26
processing 

processing time is 5.14546
frame: 46
processing time is 5.19764
frame: 47
processing time is 5.17103
frame: 48
processing time is 5.16063
frame: 49
processing time is 5.15069
[[[587. 676.]
  [558. 578.]
  [568. 485.]
  ...
  [641. 421.]
  [590. 443.]
  [597. 364.]]

 [[585. 671.]
  [560. 577.]
  [569. 486.]
  ...
  [642. 423.]
  [599. 457.]
  [597. 365.]]

 [[591. 664.]
  [558. 562.]
  [569. 481.]
  ...
  [645. 424.]
  [621. 475.]
  [598. 365.]]

 ...

 [[641. 678.]
  [652. 589.]
  [659. 500.]
  ...
  [702. 426.]
  [722. 468.]
  [670. 365.]]

 [[644. 678.]
  [664. 590.]
  [664. 499.]
  ...
  [707. 428.]
  [732. 467.]
  [674. 365.]]

 [[648. 675.]
  [673. 587.]
  [666. 495.]
  ...
  [711. 426.]
  [735. 468.]
  [674. 365.]]]
current video: 24.mp4
50.0
frame: 0
processing time is 5.18163
frame: 1
processing time is 5.16256
frame: 2
processing time is 5.15626
frame: 3
processing time is 5.15807
frame: 4
processing time is 5.20223
frame: 5
processing time is 5.16843
frame: 6
processing time

processing time is 5.09157
frame: 36
processing time is 5.12631
frame: 37
processing time is 5.11179
frame: 38
processing time is 5.12363
frame: 39
processing time is 5.09767
frame: 40
processing time is 5.10954
frame: 41
processing time is 5.09200
frame: 42
processing time is 5.10024
frame: 43
processing time is 5.10031
frame: 44
processing time is 5.11397
frame: 45
processing time is 5.12101
frame: 46
processing time is 5.15071
frame: 47
processing time is 5.12447
frame: 48
processing time is 5.07229
frame: 49
processing time is 5.07611
[[[597. 683.]
  [607. 589.]
  [614. 490.]
  ...
  [683. 422.]
  [686. 474.]
  [634. 359.]]

 [[597. 683.]
  [608. 587.]
  [616. 488.]
  ...
  [684. 422.]
  [689. 475.]
  [636. 358.]]

 [[598. 682.]
  [608. 587.]
  [618. 489.]
  ...
  [684. 421.]
  [690. 475.]
  [637. 357.]]

 ...

 [[617. 681.]
  [613. 585.]
  [613. 488.]
  ...
  [698. 415.]
  [694. 456.]
  [637. 358.]]

 [[612. 678.]
  [605. 581.]
  [611. 488.]
  ...
  [701. 416.]
  [707. 418.]
  [63

processing time is 5.13343
frame: 23
processing time is 5.13330
frame: 24
processing time is 5.14368
frame: 25
processing time is 5.11784
frame: 26
processing time is 5.14505
frame: 27
processing time is 5.13949
frame: 28
processing time is 5.11919
frame: 29
processing time is 5.14284
frame: 30
processing time is 5.12553
frame: 31
processing time is 5.12108
frame: 32
processing time is 5.14450
frame: 33
processing time is 5.13040
frame: 34
processing time is 5.12154
frame: 35
processing time is 5.12613
frame: 36
processing time is 5.12254
frame: 37
processing time is 5.14743
frame: 38
processing time is 5.13146
frame: 39
processing time is 5.14773
frame: 40
processing time is 5.12461
frame: 41
processing time is 5.14905
frame: 42
processing time is 5.11850
frame: 43
processing time is 5.14546
frame: 44
processing time is 5.13447
frame: 45
processing time is 5.11915
frame: 46
processing time is 5.17378
frame: 47
processing time is 5.12715
frame: 48
processing time is 5.11428
frame: 49
p

processing time is 5.12983
frame: 6
processing time is 5.13014
frame: 7
processing time is 5.14991
frame: 8
processing time is 5.12282
frame: 9
processing time is 5.13578
frame: 10
processing time is 5.12600
frame: 11
processing time is 5.11489
frame: 12
processing time is 5.12101
frame: 13
processing time is 5.11854
frame: 14
processing time is 5.10835
frame: 15
processing time is 5.13740
frame: 16
processing time is 5.12083
frame: 17
processing time is 5.13772
frame: 18
processing time is 5.13684
frame: 19
processing time is 5.11356
frame: 20
processing time is 5.12384
frame: 21
processing time is 5.11719
frame: 22
processing time is 5.12606
frame: 23
processing time is 5.10444
frame: 24
processing time is 5.13274
frame: 25
processing time is 5.13369
frame: 26
processing time is 5.14027
frame: 27
processing time is 5.12609
frame: 28
processing time is 5.14544
frame: 29
processing time is 5.13008
frame: 30
processing time is 5.15437
frame: 31
processing time is 5.13736
frame: 32
proce

...

processing time is 5.11198
frame: 23
processing time is 5.10799
frame: 24
processing time is 5.10415
frame: 25
processing time is 5.10169
frame: 26
processing time is 5.11380
frame: 27
processing time is 5.09647
frame: 28
processing time is 5.12160
frame: 29
processing time is 5.10224
frame: 30
processing time is 5.13969
frame: 31
processing time is 5.10861
frame: 32
processing time is 5.10365
frame: 33
processing time is 5.09858
frame: 34
processing time is 5.13368
frame: 35
processing time is 5.12345
frame: 36
processing time is 5.09004
frame: 37
processing time is 5.12573
frame: 38
processing time is 5.12041
frame: 39
processing time is 5.14125
frame: 40
processing time is 5.13668
frame: 41
processing time is 5.12447
frame: 42
processing time is 5.11686
frame: 43
processing time is 5.10877
frame: 44
processing time is 5.16557
frame: 45
processing time is 5.13430
frame: 46
processing time is 5.12985
frame: 47
processing time is 5.12870
frame: 48
processing time is 5.14206
frame: 49
p

processing time is 5.10882
frame: 6
processing time is 5.10492
frame: 7
processing time is 5.09900
frame: 8
processing time is 5.10843
frame: 9
processing time is 5.11018
frame: 10
processing time is 5.13142
frame: 11
processing time is 5.12279
frame: 12
processing time is 5.10171
frame: 13
processing time is 5.08924
frame: 14
processing time is 5.09198
frame: 15
processing time is 5.10512
frame: 16
processing time is 5.11660
frame: 17
processing time is 5.11731
frame: 18
processing time is 5.11753
frame: 19
processing time is 5.11079
frame: 20
processing time is 5.10667
frame: 21
processing time is 5.11124
frame: 22
processing time is 5.09695
frame: 23
processing time is 5.09890
frame: 24
processing time is 5.09448
frame: 25
processing time is 5.10891
frame: 26
processing time is 5.11669
frame: 27
processing time is 5.10928
frame: 28
processing time is 5.10335
frame: 29
processing time is 5.10882
frame: 30
processing time is 5.12456
frame: 31
processing time is 5.09929
frame: 32
proce

processing time is 5.10397
frame: 48
processing time is 5.10665
frame: 49
processing time is 5.11532
[[[589. 657.]
  [526. 591.]
  [588. 554.]
  ...
  [643. 509.]
  [589. 497.]
  [606. 447.]]

 [[588. 656.]
  [524. 592.]
  [585. 554.]
  ...
  [644. 507.]
  [590. 499.]
  [608. 447.]]

 [[584. 657.]
  [520. 590.]
  [583. 552.]
  ...
  [644. 507.]
  [590. 501.]
  [612. 446.]]

 ...

 [[567. 667.]
  [516. 592.]
  [564. 543.]
  ...
  [667. 557.]
  [673. 614.]
  [637. 476.]]

 [[566. 666.]
  [505. 598.]
  [563. 541.]
  ...
  [657. 554.]
  [659. 610.]
  [638. 475.]]

 [[565. 664.]
  [502. 601.]
  [562. 540.]
  ...
  [653. 553.]
  [654. 610.]
  [638. 476.]]]
current video: 99.mp4
50.0
frame: 0
processing time is 5.10627
frame: 1
processing time is 5.10539
frame: 2
processing time is 5.10772
frame: 3
processing time is 5.11287
frame: 4
processing time is 5.10704
frame: 5
processing time is 5.10451
frame: 6
processing time is 5.13203
frame: 7
processing time is 5.09429
frame: 8
processing time i

processing time is 5.11107
frame: 37
processing time is 5.09702
frame: 38
processing time is 5.12273
frame: 39
processing time is 5.11304
frame: 40
processing time is 5.10457
frame: 41
processing time is 5.10929
frame: 42
processing time is 5.11612
frame: 43
processing time is 5.09546
frame: 44
processing time is 5.13126
frame: 45
processing time is 5.10099
frame: 46
processing time is 5.13298
frame: 47
processing time is 5.12241
frame: 48
processing time is 5.10523
frame: 49
processing time is 5.10855
[[[563. 663.]
  [589. 580.]
  [604. 493.]
  ...
  [  0.   0.]
  [  0.   0.]
  [643. 376.]]

 [[564. 662.]
  [591. 580.]
  [604. 495.]
  ...
  [  0.   0.]
  [  0.   0.]
  [644. 375.]]

 [[566. 662.]
  [593. 581.]
  [604. 494.]
  ...
  [  0.   0.]
  [  0.   0.]
  [643. 374.]]

 ...

 [[556. 664.]
  [572. 578.]
  [599. 500.]
  ...
  [591. 442.]
  [538. 425.]
  [594. 390.]]

 [[557. 664.]
  [574. 579.]
  [597. 497.]
  ...
  [574. 438.]
  [524. 415.]
  [589. 386.]]

 [[557. 664.]
  [577. 580.

processing time is 5.10684
frame: 23
processing time is 5.10696
frame: 24
processing time is 5.11363
frame: 25
processing time is 5.09409
frame: 26
processing time is 5.10989
frame: 27
processing time is 5.11622
frame: 28
processing time is 5.11368
frame: 29
processing time is 5.12378
frame: 30
processing time is 5.11758
frame: 31
processing time is 5.12854
frame: 32
processing time is 5.12011
frame: 33
processing time is 5.10781
frame: 34
processing time is 5.12829
frame: 35
processing time is 5.13242
frame: 36
processing time is 5.11491
frame: 37
processing time is 5.16425
frame: 38
processing time is 5.11379
frame: 39
processing time is 5.09783
frame: 40
processing time is 5.14825
frame: 41
processing time is 5.12878
frame: 42
processing time is 5.13381
frame: 43
processing time is 5.13852
frame: 44
processing time is 5.14840
frame: 45
processing time is 5.14330
frame: 46
processing time is 5.12560
frame: 47
processing time is 5.14678
frame: 48
processing time is 5.13733
frame: 49
p

In [37]:
ls popping99_csv/

100.csv  13.csv  23.csv  33.csv  43.csv  53.csv  63.csv  73.csv  83.csv  92.csv
100.mat  13.mat  23.mat  33.mat  43.mat  53.mat  63.mat  73.mat  83.mat  92.mat
101.csv  14.csv  24.csv  34.csv  44.csv  54.csv  64.csv  74.csv  84.csv  93.csv
101.mat  14.mat  24.mat  34.mat  44.mat  54.mat  64.mat  74.mat  84.mat  93.mat
102.csv  15.csv  25.csv  35.csv  45.csv  55.csv  65.csv  75.csv  85.csv  94.csv
102.mat  15.mat  25.mat  35.mat  45.mat  55.mat  65.mat  75.mat  85.mat  94.mat
103.csv  16.csv  26.csv  36.csv  46.csv  56.csv  66.csv  76.csv  86.csv  95.csv
103.mat  16.mat  26.mat  36.mat  46.mat  56.mat  66.mat  76.mat  86.mat  95.mat
104.csv  17.csv  27.csv  37.csv  47.csv  57.csv  67.csv  77.csv  87.csv  96.csv
104.mat  17.mat  27.mat  37.mat  47.mat  57.mat  67.mat  77.mat  87.mat  96.mat
105.csv  18.csv  28.csv  38.csv  48.csv  58.csv  68.csv  78.csv  88.csv  97.csv
105.mat  18.mat  28.mat  38.mat  48.mat  58.mat  68.mat  78.mat  88.mat  97.mat
106.csv  19.csv  29.csv  39.csv  49.csv 

In [ ]:
# Copy the generated file back to G-drive
!cp -r /content/popping99_csv /drive/popping99_csv

In [ ]:
!mkdir popping99_csv

In [ ]:
cd /content/